In [ ]:

%load_ext autoreload 
%autoreload 2
%matplotlib widget
%matplotlib inline
import multiple_planets_gas_acc as code_gas
import functions_pebble_accretion as peb
import numpy as np
from scipy import integrate
from functions import *
from functions_pebble_accretion import *
import functions_plotting as plot
from matplotlib import cm, ticker
from matplotlib import colors
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import astropy.units as u
import numpy as np
import pandas as pd
import matplotlib.cm as cm
from tqdm.notebook import tqdm
import matplotlib.gridspec as gridspec
import cProfile 
from matplotlib.ticker import ScalarFormatter, LogFormatter, LogLocator
import matplotlib.gridspec as gridspec
import matplotlib.patches as patch
from matplotlib.offsetbox import AnchoredText
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
import matplotlib.lines as mlines 
from matplotlib.ticker import MultipleLocator, AutoMinorLocator

color = mpl.colormaps["YlOrRd"].reversed()(np.linspace(0, 0.7, code_gas.sim_params.nr_planets))


In [ ]:
# disc parameters
params_dict = {'St_const': None, 
                'M_dot_star': None,
                'v_frag':(1 * u.m/u.s).to(u.au/u.Myr).value,
                'alpha_z': 1e-4, 
                'alpha_frag': 1e-4, 
                'iceline_alpha_change': False,
                'iceline_flux_change': False,
                }

t_initial = 0.2
a_p0 = np.geomspace(1,1, num = 1)
t0= [t_initial] * np.ones(len(a_p0))# warning, this also goes in the initial conditions when doing mulitple planets otherwise it won't work
t_in = 0.2
t_fin = 3
sim_params_dict = {'N_step': 1000,
                'a_p0':a_p0,
                't0':t0,
                't_in':t_in,
                't_fin':t_fin
                }

params_ida_ext_2 = code_gas.Params(**params_dict, H_r_model='Ida_mixed', epsilon_el=1, epsilon_heat=1, alpha =1e-2)
# params_ida_ext_3 = code_gas.Params(**params_dict, H_r_model='Ida_mixed', epsilon_el=1, epsilon_heat=1, alpha =1e-3)

# params_lam_ext_2 = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5, alpha =1e-2)
# params_lam_ext_3 = code_gas.Params(**params_dict, H_r_model='Lambrechts_mixed', epsilon_el=1e-2, epsilon_heat=0.5, alpha =1e-3)


# m0_lam_ext_2 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_lam_ext_2), params_lam_ext_2)
# sim_params_lam_ext_2 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_lam_ext_2)
# m0_lam_ext_3 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_lam_ext_3), params_lam_ext_3)
# sim_params_lam_ext_3 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_lam_ext_3)

m0_ida_ext_2 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_ida_ext_2), params_ida_ext_2)
sim_params_ida_ext_2 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_ida_ext_2)
# m0_ida_ext_3 = M0_pla(a_p0, t0, sigma_gas_steady_state(a_p0, t0, params_ida_ext_3), params_ida_ext_3)
# sim_params_ida_ext_3 = code_gas.SimulationParams(**sim_params_dict, m0 = m0_ida_ext_3)

peb_acc = peb.PebbleAccretion(simplified_acc=False)
gas_acc = peb.GasAccretion()

# sim_lam_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_lam_ext_2, sim_params=sim_params_lam_ext_2)
# sim_lam_ext_3 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_lam_ext_3, sim_params=sim_params_lam_ext_3)

sim_ida_ext_2 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_2, sim_params=sim_params_ida_ext_2)
#sim_ida_ext_3 = code_gas.simulate_euler(migration=True, filtering=True, peb_acc=peb_acc, gas_acc=gas_acc, params=params_ida_ext_3, sim_params=sim_params_ida_ext_3)


In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(6, 6))
axs.loglog(sim_ida_ext_2.time, sim_ida_ext_2.sigma_gas[0])
print(sim_ida_ext_2.sigma_gas[0])

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(5, 5))
pos = np.geomspace(1e-2, 100, num = 10)
t = np.geomspace(1e-1, 10, num = 10)
t0 = 0.1
params = code_gas.Params(**params_dict)
axs.loglog(pos, H_R_irr(pos, params_ida_ext_2))

In [ ]:
erg_s_to_au_M_E_Myr = (1*u.erg/u.s).to(u.au**2*u.M_earth/u.Myr**3).value
L = (const.L_sun.cgs).to(u.au**2*u.M_earth/u.Myr**3)
M = (const.M_sun).to(u.M_earth)
r = 1*u.au
print((const.L_sun.cgs).to(u.au**2*u.M_earth/u.Myr**3))
print((const.M_sun).to(u.M_earth))
params = code_gas.Params(**params_dict, star_luminosity= L, star_mass = M)
print( "in au",0.024*(L)**(1/7)*(M)**(-4/7)*(r)**(2/7))
print(H_R_irr(1, params))

In [ ]:
fig,axs = plt.subplots(2,2, figsize = (18,18))
label = ["no filtering", "filtering"]
title = [ [ "Lambrechts disc, 1e-3", "Lambrechts disc, 1e-2"], ["Liu disc, 1e-3", "Liu disc 1e-2"] ]


# plot.plot_growth_track_timescale(fig, axs[0], sim_filter_irr, params_irr, sim_params_irr, migration = True,
#                                     cmap = mpl.colormaps["inferno"].reversed(), add_ylabel=True , 
#                                         add_cbar= True , nofilter=True if i ==0 else False
#                                         )

# plot.plot_growth_track_timescale(fig, axs[0,0], sim_lam_ext_3, params_lam_ext_3, sim_params_lam_ext_3, migration = True,
#                                 cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
#                                     add_cbar=False, nofilter=False
#                                     )

# plot.plot_growth_track_timescale(fig, axs[0,1], sim_lam_ext_2, params_lam_ext_2, sim_params_lam_ext_2, migration = True,
#                                 cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
#                                     add_cbar=True, nofilter=False
#                                     )

# plot.plot_growth_track_timescale(fig, axs[1,0], sim_ida_ext_3, params_ida_ext_3, sim_params_ida_ext_3, migration = True,
#                                 cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
#                                     add_cbar=False, nofilter=False
#                                     )

plot.plot_growth_track_timescale(fig, axs[1,1], sim_ida_ext_2, params_ida_ext_2, sim_params_ida_ext_2, migration = True,
                                cmap = mpl.colormaps["inferno"].reversed(), add_ylabel= False, 
                                    add_cbar=True, nofilter=False
                                    )



for i in range(2):
    for j in range(2):
        axs[i,j].tick_params(axis = "both", which = "major", direction = 'in', size = 15)
        axs[i,j].tick_params(axis = "both", which = "minor", direction = 'in', size = 10)
        axs[i,j].set_xlim(4e-3, 1e2)
        axs[i,j].set_title(title[i][j], fontsize = 20)
        plot.kepler11(axs[i,j], color = 'black')
        plot.solar_system(axs[i,j], color = 'green')


plt.savefig("figures/1planet_ida_lam_2_3", bbox_inches='tight')